# YOLO Wall Detection Training - CPU/GPU Auto-Detect
**Arc Hackathon Project - Custom Wall Element Detection**

In [ ]:
# Cell 1: Install Dependencies (GPU)
!pip install ultralytics scikit-learn
print("✅ Dependencies installed")

In [ ]:
# Cell 2: Extract and Setup Training Data
import zipfile
import os
import shutil
from sklearn.model_selection import train_test_split

# Extract dataset
with zipfile.ZipFile('arc_training_data.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

print("✅ Dataset extracted")

# Check what we have
train_images = os.listdir('images/train')
train_labels = os.listdir('labels/train')

print(f"Total images: {len(train_images)}")
print(f"Total labels: {len(train_labels)}")

# Create validation split (20% for validation)
train_imgs, val_imgs = train_test_split(train_images, test_size=0.2, random_state=42)

# Create val directories
os.makedirs('images/val', exist_ok=True)
os.makedirs('labels/val', exist_ok=True)

# Move 20% to validation
for img in val_imgs:
    # Move image
    shutil.move(f'images/train/{img}', f'images/val/{img}')
    # Move corresponding label
    label_name = img.replace('.jpg', '.txt')
    if label_name in train_labels:
        shutil.move(f'labels/train/{label_name}', f'labels/val/{label_name}')

print(f"Training images: {len(os.listdir('images/train'))}")
print(f"Validation images: {len(os.listdir('images/val'))}")

# Create data.yaml file with absolute paths
current_dir = os.getcwd()

data_yaml_content = f"""
train: {current_dir}/images/train
val: {current_dir}/images/val

nc: 5
names: ['mirror', 'thermostat', 'vent', 'wall_socket', 'window_box']
"""

with open('data.yaml', 'w') as f:
    f.write(data_yaml_content)

print(f"Data paths: {current_dir}/images/train, {current_dir}/images/val")
print("✅ Dataset split and data.yaml created")

In [ ]:
# Cell 3: Check System
import torch
import numpy as np
print(f"PyTorch version: {torch.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")
print("✅ System check complete")

In [ ]:
# Cell 4: Train YOLO Model
from ultralytics import YOLO

# Load YOLOv8 nano model
model = YOLO('yolov8n.pt')
print("✅ Model loaded")

# Auto-detect device and optimize settings
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Training on: {device}')

# Optimize settings based on device
if device == 'cuda':
    epochs, batch, imgsz, workers = 100, 16, 640, 4
    name = 'gpu_training'
else:
    epochs, batch, imgsz, workers = 50, 8, 416, 2
    name = 'cpu_training'
    print('⚠️  CPU training will take 6-8 hours!')

results = model.train(
    data='data.yaml',
    epochs=epochs,
    batch=batch,
    imgsz=imgsz,
    device=device,
    workers=workers,
    patience=15,
    save_period=10,
    project='wall_detection',
    name=name
)

print("✅ Training completed!")

In [ ]:
# Cell 5: Test and Show Results
import glob

# Load best model
# Load model (auto-detect training type)
import glob
model_path = glob.glob('wall_detection/*/weights/best.pt')[0]
best_model = YOLO(model_path)

# Validate and get metrics
print('📊 Running validation...')
metrics = best_model.val(data='data.yaml')

# Display results
print('\n🎯 ACCURACY RESULTS:')
print(f'mAP50: {metrics.box.map50:.3f} ({metrics.box.map50*100:.1f}%)')
print(f'mAP50-95: {metrics.box.map:.3f} ({metrics.box.map*100:.1f}%)')
print(f'Precision: {metrics.box.mp:.3f} ({metrics.box.mp*100:.1f}%)')
print(f'Recall: {metrics.box.mr:.3f} ({metrics.box.mr*100:.1f}%)')

print('\n✅ Validation completed')

In [ ]:
# Cell 6: Save Model
import shutil

# Copy model for use
# Copy model (auto-detect training type)
import glob
model_path = glob.glob('wall_detection/*/weights/best.pt')[0]
device_type = 'gpu' if torch.cuda.is_available() else 'cpu'
shutil.copy(model_path, f'wall_model_{device_type}.pt')

device_type = 'gpu' if torch.cuda.is_available() else 'cpu'
print(f"✅ Model ready: wall_model_{device_type}.pt")
print("Copy this file to your Arc project folder")
print("\n🏆 TRAINING COMPLETE!")
print("\n🎯 Your model detects:")
print("  - mirror")
print("  - thermostat")
print("  - vent")
print("  - wall_socket")
print("  - window_box")
print("\n🚀 Ready for Arc hackathon!")